# 1. LLM
單純的 text completion models

- **`__call__`** : Text -> Text
- **`generate`** : batch of text -> batch of text and rich information

In [1]:
from pprint import pprint
from langchain_setup import OpenAI

llm = OpenAI()

print("===== __call__ =====")
text = "你是一個非常厲害的唐詩詩人，同時你對金融業十分了解。\n\n請寫一首關於金融業的八句七言律詩："
print(llm(text), end='\n\n')

print("===== generate =====")
text1 = "Hello World"
text2 = "How do you do? I am fine thank you."
result = llm.generate([text1, text2])
print(type(result))
pprint(result.__dict__)

===== __call__ =====


金錢本身無定形，
財富資源可輕凌。
貨幣起萌芽自然，
發展投資市場可稱雄。

投資動態引爆火，
決策依循精確計算。
行情風險須勤慎，
避免失敗千萬叮嚀。

投資詞語唱出詩，
把握時機細思量。
做出選擇最重要，
掌握金融利潤

===== generate =====
<class 'langchain.schema.output.LLMResult'>
{'generations': [[Generation(text='\n\nHello! My name is Adam and I am new to programming. I am excited to learn about coding and see what I can do with it.', generation_info={'finish_reason': 'stop', 'logprobs': None})],
                 [Generation(text="\n\nIt's nice to meet you. It's nice to meet you too.", generation_info={'finish_reason': 'stop', 'logprobs': None})]],
 'llm_output': {'model_name': 'text-davinci-003',
                'token_usage': {'completion_tokens': 48,
                                'prompt_tokens': 13,
                                'total_tokens': 61}},
 'run': [RunInfo(run_id=UUID('4c28ecdf-c3d9-4fd8-8bfa-1c06c07acdff')),
         RunInfo(run_id=UUID('ee9bda3f-5503-40f8-a66d-5f762e81178a'))]}


# 2. Chat Models
Chat models 是針對對話特別 tune 過的 message completion models
- **`__call__`** : Message"s" -> message
- **`generate`** : batch of message"s" -> batch of message and rich information

In [1]:
from pprint import pprint
from langchain.schema import (
    # 各種不同的角色（role）
    AIMessage, # AI
    HumanMessage, # 人類
    SystemMessage, # 設定
    ChatMessage, # 任何話者
    FunctionMessage, # 其他工具的 API(?)
)
from langchain_setup import ChatOpenAI

chat_model = ChatOpenAI()

print("===== __call__ =====")
text = "你是一個非常厲害的唐詩詩人，同時你對金融業十分了解。\n\n請寫一首關於金融業的八句七言律詩："
messages = [
    SystemMessage(content="你是一個非常厲害的唐詩詩人，同時你對金融業十分了解。"),
    HumanMessage(content="請寫一首關於金融業的八句七言律詩：")
]
response_message = chat_model(messages)
print(type(response_message))
pprint(response_message.__dict__)

print("\n===== generate =====")
batch_messages = [
    [
        SystemMessage(content="你是一個非常厲害的唐詩詩人，同時你對金融業十分了解。"),
        HumanMessage(content="請寫一首關於金融業的八句七言律詩："),
    ],
    [
        HumanMessage(content="How do you do?"),
        AIMessage(content="I am fine thank you."),
        HumanMessage(content="hontouni?"),
    ]
]
result = chat_model.generate(batch_messages)
print(type(result))
pprint(result.__dict__)

===== __call__ =====
<class 'langchain.schema.messages.AIMessage'>
{'additional_kwargs': {},
 'content': '金融融通經濟網，\n'
            '財富流轉著利益。\n'
            '股市震盪壯志氣，\n'
            '銀行興衰現蹤跡。\n'
            '風險潛藏投資間，\n'
            '理財智慧永綿延。\n'
            '金融業舞台繁華，\n'
            '創富與損失同纏繞。',
 'example': False,
 'type': 'ai'}

===== generate =====
<class 'langchain.schema.output.LLMResult'>
{'generations': [[ChatGeneration(text='山河萬里展宏圖，財富從此散芬芳。\n股市波濤起浩浩，投資風險勿輕忽。\n貨幣流通如江河，利率漲落隨波逐。\n金融巨擘談策略，智慧經營創新高。\n風險管理與監管，穩健發展擁光芒。\n金錢乃魔亦為神，賢者智慧方能當。', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='山河萬里展宏圖，財富從此散芬芳。\n股市波濤起浩浩，投資風險勿輕忽。\n貨幣流通如江河，利率漲落隨波逐。\n金融巨擘談策略，智慧經營創新高。\n風險管理與監管，穩健發展擁光芒。\n金錢乃魔亦為神，賢者智慧方能當。'))],
                 [ChatGeneration(text='As an AI, I don\'t have feelings, so I don\'t experience things like "fine" or "not fine" in the same way humans do. However, I am here to assist you with any questions or tasks you have. How can I help you today?', generation_info={'fini

# 3. Stop

有時候語言模型 (LLM) 會有產生過長的回答、在回答後加上多餘的解釋、不等待使用者回覆直接繼續講下去等等情況。其中一個解決方法是當模型將產出某些字時強制停止。

In [2]:
response_message = chat_model(messages, stop=["。", "，"]) # 遇到這些字就停止
response_message.content

'金融興起繁華綻'

# 4. Generation Parameters

- Temperature (溫度，創造力): 0 ~ 1
  - 越低越 deterministic, 越高越 random 
  - $$p_i = \frac{e^{x_i/\tau}}{\sum_{j=1}^n e^{x_j/\tau}}$$
  - 1 時為原本的 probability distribution, 越接近 0 其 prob. dist. 越 skewed
- Top p: 0 ~ 1
  - 還是有機率抽到不適合的字詞，為了避免這種情況我們可以選擇絕對不要選一些機率小的字詞
  - 候選字詞由機率高到機率低一個個選，並累加其機率到設定的 p 為止，之後的候選字詞絕對不選，只會從剛才選的候選裡面 sample
- presence/frequency penalty
  - 假設 $f_w$ 為此次生成中，字詞 w 出現的次數
  - $logit(w) = logit(w) - f_w * frequency\_penalty - (f_w > 0) * presence\_penalty$
  - frequency penalty 主要是用來減少重複字詞
  - presence penalty 主要是想要開新主題
  - 兩個其實非常類似
- logit bias
  - $logit(w) += logit\_bias(w)$
  - e.g.可以設一個很大的負值如 -100, 來保證某些字詞絕對不會被產生
  - e.g.可以設一個小小的正值，來讓某些字詞更容易被產生
- stop
  - 遇到某些字詞後強制停止生成
- logprobs
  - 可以回傳前 logprobs 個機率高的字詞的 log probabilities
  - langchain 好像沒有 explicit support

## 5. LLM 和 Chat Models 的互換性
雖然一般語言模型和對話式語言模型有不同的輸入和輸出形式，但對很多應用來說需要的就是一個語言模型，不一定特別要一般語言模型或對話式語言模型。

因此 Langchain 幫兩種模型創造了相同的介面，使其在大部分的地方都能夠互換。
- `predict`: text -> text
- `predict_messages`: messages -> message

In [1]:
from langchain_setup import OpenAI, ChatOpenAI
from langchain.schema import AIMessage, SystemMessage, HumanMessage
from langchain.schema import get_buffer_string

messages = [
    SystemMessage(content="你是一個非常厲害的唐詩詩人，同時你對金融業十分了解。"),
    HumanMessage(content="請寫一首關於金融業的八句七言律詩：")
]

text = "你是一個非常厲害的唐詩詩人，同時你對金融業十分了解。\n\n請寫一首關於金融業的八句七言律詩："

In [2]:
llm = OpenAI()

print("把 LLM 當成 Chat 模型來用:")
message = llm.predict_messages(messages=messages)
print(type(message))
print(message.content)

print("\n\n實際上給 LLM 的 Prompt 會長這樣:")
prompt = get_buffer_string(messages=messages)
print(prompt)

把 LLM 當成 Chat 模型來用:
<class 'langchain.schema.messages.AIMessage'>


金財慾望油然而生，
風輕雲淡破空間行；
海嘯起搖落無定訣，
做夢不想到漩渦中。

銀嶺煙霧遮不住視，
財海萬象入眼睛；
堅忍抗拒貪嗔痴心，
智慧總是明亮的光。

把握時機自由行動，
現實沒有捷徑可走；
把握良機智慧把握，



實際上給 LLM 的 Prompt 會長這樣:
System: 你是一個非常厲害的唐詩詩人，同時你對金融業十分了解。
Human: 請寫一首關於金融業的八句七言律詩：


In [3]:
chat_model = ChatOpenAI()

print("把 Chat model 模型當 LLM 用:")
print(chat_model.predict(text=text), end='\n\n')

print("內部的 messages 其實長這樣:")
print([HumanMessage(content=text)])

把 Chat model 模型當 LLM 用:
金融巨浪舞逐潮，投資狂潮漲澎湃。
財富蕩漾心總亂，風雨驚濤令人敗。
股市波瀾驚心神，利率猛烈起伏傷。
投資者如遊泳者，須懂風向與機關。
創新科技衝擊力，資本市場百花香。
金融業如江湖客，智慧與風險要相當。

內部的 messages 其實長這樣:
[HumanMessage(content='你是一個非常厲害的唐詩詩人，同時你對金融業十分了解。\n\n請寫一首關於金融業的八句七言律詩：')]
